In [20]:
import seaborn as sns
df = sns.load_dataset('tips')


In [21]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [22]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [23]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [24]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [25]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [26]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [27]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [28]:
df.time.unique()

array([0, 1])

In [29]:
X = df.drop(labels=['time'],axis = 1)
y = df.time

In [30]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [31]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [32]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [33]:
y.unique()

array([0, 1])

In [34]:
type(y)

pandas.core.series.Series

In [36]:
import pandas as pd

In [38]:
pd.DataFrame(y).head()

,time
0,0
1,0
2,0
3,0
4,0


In [45]:
y.value_counts()

0    176
1     68
Name: time, dtype: int64

In [111]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.20)

In [48]:
##Learn Pipeline below to automate our process

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ##Handle Missing Values
from sklearn.preprocessing import StandardScaler  ##for feature scaling
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder  ##Categorical to numerical conversion


In [49]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [52]:
df.dtypes

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time             int64
size             int64
dtype: object

In [112]:
list((df.dtypes[(df.dtypes == 'category')]).to_dict())

['sex', 'smoker', 'day']

In [113]:
list((df.dtypes[(df.dtypes != 'category')]).to_dict())

['total_bill', 'tip', 'time', 'size']

In [114]:
categorical_cols = ['sex', 'smoker', 'day']
numerical_cols = ['total_bill', 'tip', 'size']


In [115]:
##feature EngineeringAutomation
## Numerical Pipelines
num_pipeline = Pipeline(
    steps = [
            ('imputer',SimpleImputer(strategy='median')), ##Misiing Values
            ('scaler',StandardScaler()) ##Feature Scaling
            ]
)

cat_pipeline = Pipeline(
      steps=[
            ('imputer',SimpleImputer(strategy= 'most_frequent')), ##Handle missing values
            ('onehotencoder',OneHotEncoder()) ##Categorical features to numerical
            ]
)

In [116]:
preprocessor = ColumnTransformer(
    [
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
    ]
)

In [117]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [120]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [121]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [122]:
##Model training Automation

models = { 'Random Forest' : RandomForestClassifier(),
          'Logistic Regression' : LogisticRegression(),
          'Decision Tree': DecisionTreeClassifier()
}

In [123]:
from sklearn.metrics import accuracy_score

In [130]:
def evaluate_model(X_train,X_test,y_train,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]

        ##Train Model
        model.fit(X_train,y_train)

        ##Predicting Testing Data
        y_test_pred = model.predict(X_test)
        
        ##Get Accuracy for test data prediction
        
        test_model_score = accuracy_score(y_test,y_test_pred)
        
        report[list(models.keys())[i]] = test_model_score
        
    return report

In [131]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [132]:
classifier = RandomForestClassifier()

In [133]:
##Hyperparameter Tuning
params = {'max_depth' : [3,5,10,None],
          'n_estimators':[100,200,300],
          'criterion':['gini','entropy']
         }

In [134]:
from sklearn.model_selection import RandomizedSearchCV

In [135]:
cv = RandomizedSearchCV(classifier,param_distributions=params,scoring = 'accuracy',cv = 5,verbose = 3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.949 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=300;, score=0.949 total time=   0.6s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=300;, score=0.974 total time=   0.6s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=300;, score=0.923 total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=300;, score=0.949 total time=   0.6s
[CV 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [136]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 10, 'criterion': 'entropy'}